In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dropout, Dense, Flatten, Input, Layer, concatenate
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
import gc
import imageio
from os import listdir
from os.path import isfile, join
from tensorflow import keras
import shutil
import os
import PIL.Image
import glob

In [9]:
# Loading csv files
train_csv = pd.read_csv('/Users/lovedeepsingh/My Files/Projects/petfinder-pawpularity-score/train.csv',index_col=[0])
test_csv = pd.read_csv('/Users/lovedeepsingh/My Files/Projects/petfinder-pawpularity-score/test.csv',index_col=[0])

In [10]:
# Dividing the data
x_train = train_csv.iloc[:,:-1]
y = train_csv.iloc[:,-1]
y = y.sort_index()
x_train = x_train.sort_index()
type(y)

pandas.core.series.Series

In [11]:
# Spliting training and validation set from csv file for xgboost model
x_train_1, x_valid, y_train, y_valid = train_test_split(x_train, y, test_size=0.4, random_state=0)

In [12]:
# Loading folders containing images 
train_images_folder = "/Users/lovedeepsingh/My Files/Projects/petfinder-pawpularity-score/train/"
test_images_folder = "/Users/lovedeepsingh/My Files/Projects/petfinder-pawpularity-score/test/"

In [13]:
# Now building 1st model
model_12 = InceptionV3(input_shape=(100,100,3),include_top=False, weights='imagenet')
for layer in model_12.layers:
    layer.trainable=False
x = Flatten()(model_12.output)
x = Dense(24, activation='relu')(x)
x = Dense(124, activation='relu')(x)
x = Dense(64, activation='relu')(x)
x = Dense(12, activation='sigmoid', name='features')(x)
output_1 = x
model_1 = Model(model_12.input, output_1)

In [14]:
model_1.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 100, 100, 3) 0                                            
__________________________________________________________________________________________________
conv2d_94 (Conv2D)              (None, 49, 49, 32)   864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_94 (BatchNo (None, 49, 49, 32)   96          conv2d_94[0][0]                  
__________________________________________________________________________________________________
activation_94 (Activation)      (None, 49, 49, 32)   0           batch_normalization_94[0][0]     
______________________________________________________________________________________________

In [15]:
# Compiling the Model_1
model_1.compile(optimizer='adam',loss='mse')

In [16]:
# Image Data Generator
img_data_gen = ImageDataGenerator(rotation_range=40,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  shear_range=0.1,
                                  zoom_range=0.2,
                                  horizontal_flip=True,
                                  vertical_flip=True)

In [17]:
# Defining a function for getting specific images and their outputs
def listing_function(y, x_train_1, images_folder):
    expectations_from_model_1 = []
    imagees = []
    answers = []
    train_images_folder_ready = glob.glob(images_folder + '/*.jpg')
    train_images_folder_ready.sort(reverse=False)

    for imagess in train_images_folder_ready:
            m = (os.path.basename(imagess))
            verify = (m)
            verify = os.path.splitext(verify)
            verify = verify[0]
            for i in x_train_1.index:
                    if verify == i:
                            x_lt = tf.io.read_file(imagess)
                            x_lt = tf.image.decode_jpeg(x_lt, channels=3)
                            x_lt = tf.cast(x_lt, tf.float32) / 255.0
                            x_lt = tf.image.resize(x_lt, (100,100))
                            x_lt = np.array(x_lt)
                            imagees.append(x_lt)
                            
                            x_vals_final = x_train_1.loc[i]
                            x_vals_final = np.array(x_vals_final)
                            expectations_from_model_1.append(x_vals_final)
                            
                            if len(y)>=1:
                                y_final = y[i]
                                y_final = np.array(y_final)
                                answers.append([y_final])
                            else:
                                answers = []

    return expectations_from_model_1, imagees, answers

In [18]:
# Getting values from the defined function
expectations_from_model_1, imagees, answers = listing_function(y_train, x_train_1, train_images_folder)

In [19]:
# Converting the values into an array
imagees = np.array(imagees)

In [20]:
del answers
gc.collect()

0

In [21]:
# Training model_1
recall = keras.callbacks.EarlyStopping(monitor='loss',patience=10,restore_best_weights=True)
model_1.fit(img_data_gen.flow(imagees, expectations_from_model_1,
                              batch_size=100),steps_per_epoch=len(x_train_1)//100,
                                                epochs=25, callbacks=[recall])

2021-12-05 11:04:42.376879: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-12-05 11:04:42.377015: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/25


2021-12-05 11:04:43.263985: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


59/59 [==============================] - 10s 124ms/step - loss: 0.1115
Epoch 2/25
59/59 [==============================] - 6s 103ms/step - loss: 0.0871
Epoch 3/25
59/59 [==============================] - 6s 103ms/step - loss: 0.0853
Epoch 4/25
59/59 [==============================] - 6s 103ms/step - loss: 0.0840
Epoch 5/25
59/59 [==============================] - 6s 103ms/step - loss: 0.0830
Epoch 6/25
59/59 [==============================] - 6s 103ms/step - loss: 0.0818
Epoch 7/25
59/59 [==============================] - 6s 103ms/step - loss: 0.0809
Epoch 8/25
59/59 [==============================] - 6s 103ms/step - loss: 0.0804
Epoch 9/25
59/59 [==============================] - 6s 103ms/step - loss: 0.0799
Epoch 10/25
59/59 [==============================] - 6s 103ms/step - loss: 0.0797
Epoch 11/25
59/59 [==============================] - 6s 104ms/step - loss: 0.0792
Epoch 12/25
59/59 [==============================] - 6s 105ms/step - loss: 0.0785
Epoch 13/25
59/59 [================

In [ ]:
del imagees, expectations_from_model_1
gc.collect()

# Now for training the XGBoost model

In [ ]:
# Getting test values from the function
req, valid_imgs, valid_answers = listing_function(y_valid, x_valid, train_images_folder)

In [ ]:
# Converting the valid images into an array
array_of_valid_imgs = np.array(valid_imgs)

In [ ]:
# Using the model_1 to predict the features of the pets
predict_test_vals = model_1.predict(array_of_valid_imgs)

In [ ]:
np.set_printoptions(formatter={'float_kind':'{:f}'.format})

In [ ]:
del _,valid_imgs, array_of_valid_imgs
gc.collect()

In [ ]:
def optimizing_for_result(vals):
    object_list = []
    predict_test_vals_1 = vals*10
    for items in predict_test_vals_1:
        element_list = []
        for i in items:
            if i>=3.74:
                i=1
                element_list.append(i)
            else:
                i=0
                element_list.append(i)
        object_list.append(element_list)
    object_list_1 = np.array(object_list)
    return object_list_1

In [ ]:
fitting_res = optimizing_for_result(predict_test_vals)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(req, fitting_res)

In [ ]:
# Building Model_2
xg = XGBRegressor()
xg.fit(req, valid_answers)

In [ ]:
del valid_answers, predict_test_vals, fitting_res
gc.collect()

# Getting Final Results

In [ ]:
_, test_img_final, _ = listing_function([], test_csv, test_images_folder)

In [ ]:
del _
gc.collect()

In [ ]:
array_of_test_img_final = np.array(test_img_final)

In [ ]:
final_features = model_1.predict(array_of_test_img_final)

In [ ]:
final_features_ready = optimizing_for_result(final_features)

In [ ]:
final_features_ready

In [ ]:
test_csv

In [ ]:
result_1 = xg.predict(final_features_ready)

In [ ]:
result_1

In [ ]:
finale = pd.DataFrame()
finale['Id'] = test_csv.index
finale['Pawpularity'] = result_1
finale.to_csv('submission.csv',index=False)